In [1]:
# necessary libraries
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [2]:
# load data
df_train = pd.read_csv(
    '../../../Data/favorita-grocery-sales-forecasting/train.csv', 
    usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "../../../Data/favorita-grocery-sales-forecasting/test.csv", 
    usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    "../../../Data/favorita-grocery-sales-forecasting/items.csv",
).set_index("item_nbr")

stores = pd.read_csv(
    "../../../Data/favorita-grocery-sales-forecasting/stores.csv",
).set_index("store_nbr")

In [3]:
df_train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,105574,2.564949,False
1,2016-01-01,25,105575,2.302585,False
2,2016-01-01,25,105857,1.386294,False
3,2016-01-01,25,108634,1.386294,False
4,2016-01-01,25,108701,1.098612,True


In [4]:
len(df_test)

3370464

In [5]:
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1


In [6]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [8]:
le = LabelEncoder()
items['family'] = le.fit_transform(items['family'].values)

stores['city'] = le.fit_transform(stores['city'].values)
stores['state'] = le.fit_transform(stores['state'].values)
stores['type'] = le.fit_transform(stores['type'].values)

In [9]:
df_2017 = df_train.loc[df_train.date>=pd.to_datetime('1,1,2017')]
del df_train

In [10]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)

In [29]:
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

In [31]:
del promo_2017_test, promo_2017_train

In [35]:
promo_2017

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False       False   
          2113343        False       False       False       False   
          2113914        False       False       False       False   
          2116416        False       False       False       False   

date                2017-01-09  2017-01-10  ...  2017-08-22  2017-08-23  \
store_nbr item_nbr                          ...                           
1         96995          False       False  ...       False       False   
          99197          False       False  ...       False       False   
          103520         False       False  ...       False       False   
          103665         False       False  ...       False       False   
          105574         False       False  ...       False       False   
...                        ...         ...  ...         ...         ...   
54        2109909        False       False  ...       False       False   
          2110456        False       False  ...       False       False   
          2113343        False       False  ...       False       False   
          2113914        False       False  ...        True        True   
          2116416        False       False  ...       False       False   

date                2017-08-24  2017-08-25  2017-08-26  2017-08-27  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   
...                        ...         ...         ...         ...   
54        2109909        False       False       False       False   
          2110456        False       False       False        True   
          2113343        False       False       False       False   
          2113914         True        True        True        True   
          2116416        False       False       False       False   

date                2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                  
1         96995          False       False       False       False  
          99197          False       False       False       False  
          10352

In [37]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [38]:
df_2017

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   
...                        ...         ...         ...         ...   
54        2109909          0.0    0.000000    0.000000    0.000000   
          2110456          0.0    0.000000    0.000000    0.000000   
          2113343          0.0    0.000000    0.000000    0.000000   
          2113914          0.0    0.000000    0.000000    0.000000   
          2116416          0.0    0.000000    0.000000    0.000000   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   
...                        ...         ...         ...         ...   
54        2109909     0.000000    0.000000    0.000000    0.000000   
          2110456     0.000000    0.000000    0.000000    0.000000   
          2113343     0.000000    0.000000    0.000000    0.000000   
          2113914     0.000000    0.000000    0.000000    0.000000   
          2116416     0.000000    0.000000    0.000000    0.000000   

date                2017-01-09  2017-01-10  ...  2017-08-06  2017-08-07  \
store_nbr item_nbr                          ...                           
1         96995       0.000000    0.000000  ...    1.098612    1.098612   
          99197       0.000000    0.693147  ...    0.000000    1.098612   
          103520      0.693147    0.693147  ...    0.000000    0.000000   
          103665      0.000000    2.079442  ...    0.693147    1.098612   
          105574      1.386294    2.302585  ...    0.000000    1.791759   
...                        ...         ...  ...         ...         ...   
54        2109909     0.000000    0.000000  ...    0.693147    0.693147   
          2110456     0.000000    0.000000  ...    7.203406    6.481577   
          2113343     0.000000    0.000000  ...    1.098612    0.000000   
          2113914     0.000000    0.000000  ...    2.890372    0.000000   
          2116416     0.000000    0.000000  ...    0.693147    0.693147   

date                2017-08-08  2017-08-09  2017-08-10  2017-08-11  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.693147    0.000000   
          99197       0.000000    1.098612    0.000000    0.000000   
          103520      1.386294    0.000000    1.386294    0.693147   
          103665      0.000000    2.079442    2.302585    1.098612   
          105574      2.079442    1.945910    2.397895    1.791759   
...                        ...         ...         ...         ...   
54        2109909     0.000000    1.098612    0.693147    0.000000   
          2110456     6.586172    3.218876    0.000000    0.000000   
          2113343     0.000000    0.000000    0.693147    0.000000   
          2113914     2.397895    2.397895    1.609438    0.000000   
          2116416     0.000000    0.000000    0.000000    0.693147   

date                2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                  
1         96995       0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000  
          10352

In [41]:
items = items.reindex(df_2017.index.get_level_values(1))
stores = stores.reindex(df_2017.index.get_level_values(0))

In [43]:
items

,family,class,perishable
item_nbr,,,
96995,12,1093,0
99197,12,1067,0
103520,12,1028,0
103665,5,2712,1
105574,12,1045,0
...,...,...,...
2109909,12,1074,0
2110456,3,1120,0
2113343,3,1114,0


In [45]:
stores

,city,state,type,cluster
store_nbr,,,,
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
1,18,12,3,13
...,...,...,...,...
54,5,10,2,3
54,5,10,2,3
54,5,10,2,3


In [47]:
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

In [49]:
df_2017_item

date,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,...,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15
item_nbr,,,,,,,,,,,,,,,,,,,,,
96995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.662960,7.454720,2.484907,5.950643,5.545177,8.841014,7.742402,4.969813,7.167038,7.742402
99197,0.693147,17.422746,16.604036,20.569303,16.203025,16.278613,14.775909,17.317386,14.986630,15.833927,...,3.178054,4.969813,3.178054,4.969813,2.079442,2.197225,1.386294,2.079442,0.000000,0.000000
103501,0.000000,55.868320,54.627085,42.810313,39.555298,35.717635,47.208504,47.542538,40.189274,39.200893,...,38.578235,33.531460,35.296421,35.584104,26.270815,32.776619,34.416498,36.546914,34.773173,35.512841
103520,0.000000,38.875486,35.822995,34.979211,42.252967,51.397412,49.505990,33.846832,33.336007,31.741073,...,35.630624,32.567752,47.213872,41.198030,43.569852,48.697330,47.015385,39.070042,33.798042,40.030669
103665,2.079442,56.225402,40.233610,46.138063,38.100507,49.690810,54.725492,54.286513,39.602739,35.899957,...,50.919628,41.262812,34.711732,34.095546,48.162787,50.980653,39.807856,39.016553,34.262348,35.741351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123863,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.583519,4.682131,0.000000,0.000000,2.079442
2124052,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,31.842727,18.585897,23.102412,24.877883,58.212130,66.732340,65.260886,43.252318,29.747682,30.744595
2126842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.079442,2.079442,3.583519,1.386294,1.791759


In [51]:
promo_2017_item

date,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,...,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
item_nbr,,,,,,,,,,,,,,,,,,,,,
96995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
99197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103501,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
103520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
103665,0,0,0,0,0,0,0,13,0,0,...,0,0,0,0,0,12,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2124052,0,0,0,0,0,0,0,0,0,0,...,5,11,13,18,22,20,17,15,16,23
2126842,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
df_2017_store_class = df_2017.reset_index()
df_2017_store_class['class'] = items['class'].values
df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

In [57]:
df_2017_store_class

date             2017-01-01  2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
class store_nbr                                                               
1002  1                 0.0    6.291569   11.901285    9.939627   12.817576   
      2                 0.0   27.836761   21.942946   23.265525   20.405583   
      3                 0.0   42.484074   29.286804   35.991684   29.124900   
      4                 0.0   28.353452   21.278199   22.805993   20.207757   
      5                 0.0   19.157935   15.744315   14.909440   12.177673   
...                     ...         ...         ...         ...         ...   
7780  49                0.0   14.139332    6.984716   12.542316    8.958797   
      50                0.0   14.370443    6.984716    6.356108    6.984716   
      51                0.0    4.564348    1.791759    0.693147    2.484907   
      52                0.0    0.000000    0.000000    0.000000    0.000000   
      53                0.0    6.579251    1.386294    3.988984    1.386294   

date             2017-01-06  2017-01-07  2017-01-08  2017-01-09  2017-01-10  \
class store_nbr                                                               
1002  1           10.961278   13.708549    7.454720   18.598319   12.322254   
      2           23.207544   32.629193   33.057327   18.878450   24.403008   
      3           31.628492   36.412191   32.819483   27.527092   26.893368   
      4           19.822911   24.720218   31.524160   21.634874   17.617490   
      5           12.765460   12.306750   14.427014   15.168951   12.465355   
...                     ...         ...         ...         ...         ...   
7780  49           6.761573   12.647677    6.356108    8.525161   10.450452   
      50           9.842463   12.021669    9.852615    3.871201    1.098612   
      51           0.693147    3.988984    6.327937    0.000000    0.693147   
      52           0.000000    0.000000    0.000000    0.000000    0.000000   
      53           3.178054    8.147867    4.094345    7.272398    4.564348   

date             ...  2017-08-06  2017-08-07  2017-08-08  2017-08-09  \
class store_nbr  ...                                                   
1002  1          ...    6.068426   11.966952   13.458607   11.431281   
      2          ...   28.459131   25.885741   22.351737   21.326989   
      3          ...   38.142670   27.082659   30.859413   28.748667   
      4          ...   26.304582   10.961278   17.278515   13.223041   
      5          ...   12.647677   13.969433   11.901285   15.333254   
...              ...         ...         ...         ...         ...   
7780  49         ...   10.920456    7.860185    7.965546    5.257495   
      50         ...   15.748770    7.608871    6.222576    7.377759   
      51         ...    7.390181    2.079442    1.386294    0.000000   
      52         ...    7.965546    6.068426    1.791759    3.178054   
      53         ...    4.682131    1.386294    3.583519    2.890372   

date             2017-08-10  2017-08-11  2017-08-12  2017-08-13  2017-08-14  \
class store_nbr                                                               
1002  1           13.367622    5.545177   13.628506    5.375278   18.639141   
      2           16.401095   26.828830   27.567914   25.059788   21.576931   
      3           30.232542   36.638416   31.272846   32.256757   33.608285   
      4           17.735273   19.822911   16.806560   23.187741   16.267563   
      5           13.851650   10.332669   14.416963   15.538049   11.192390   
...                     ...         ...         ...         ...         ...   
7780  49           4.852030    6.238325    6.356108   11.415533    4.718499   
      50           6.356108    7.090077    4.969813    4.499810    4.276666   
      51           0.693147    1.098612    1.098612    4.718499    0.000000   
      52           3.465736    4.969813    7.390181    7.454720    5.886104   
      53           0.693147    6.643790    7.860185    8.95

In [59]:
df_2017_promo_store_class = promo_2017.reset_index()
df_2017_promo_store_class['class'] = items['class'].values
df_2017_promo_store_class_index = df_2017_promo_store_class[['class', 'store_nbr']]
df_2017_promo_store_class = df_2017_promo_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

In [61]:
df_2017_promo_store_class

date             2017-01-01  2017-01-02  2017-01-03  2017-01-04  2017-01-05  \
class store_nbr                                                               
1002  1                   0           0           0           0           0   
      2                   0           0           0           0           0   
      3                   0           0           0           0           0   
      4                   0           0           0           0           0   
      5                   0           0           0           0           0   
...                     ...         ...         ...         ...         ...   
7780  49                  0           0           0           0           0   
      50                  0           0           0           0           0   
      51                  0           0           0           0           0   
      52                  0           0           0           0           0   
      53                  0           0           0           0           0   

date             2017-01-06  2017-01-07  2017-01-08  2017-01-09  2017-01-10  \
class store_nbr                                                               
1002  1                   0           0           0           0           0   
      2                   0           0           0           0           0   
      3                   0           0           0           0           0   
      4                   0           0           0           0           0   
      5                   0           0           0           0           1   
...                     ...         ...         ...         ...         ...   
7780  49                  0           0           0           0           0   
      50                  0           0           0           0           0   
      51                  0           0           0           0           0   
      52                  0           0           0           0           0   
      53                  0           0           0           0           0   

date             ...  2017-08-22  2017-08-23  2017-08-24  2017-08-25  \
class store_nbr  ...                                                   
1002  1          ...           1           1           1           1   
      2          ...           1           1           1           1   
      3          ...           1           1           1           1   
      4          ...           1           1           1           1   
      5          ...           0           1           1           0   
...              ...         ...         ...         ...         ...   
7780  49         ...           2           3           4           9   
      50         ...           3           2           2           5   
      51         ...           0           0           0           0   
      52         ...           0           0           0           0   
      53         ...           0           0           0           0   

date             2017-08-26  2017-08-27  2017-08-28  2017-08-29  2017-08-30  \
class store_nbr                                                               
1002  1                   1           1           1           1           1   
      2                   1           1           1           1           1   
      3                   1           1           1           1           1   
      4                   1           1           1           1           1   
      5                   1           1           1           1           1   
...                     ...         ...         ...         ...         ...   
7780  49                  5           7           3           5           5   
      50                  5           9           5           7           1   
      51                  0           0           0           0           0   
      52                  0           0           0           0           0   
      53                  0           0           0        

In [63]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [65]:
def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    X = {
        "promo_14_2017": get_timespan(promo_df, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_df, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_df, t2017, 140, 140).sum(axis=1).values,
        "promo_3_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 3).sum(axis=1).values,
        "promo_7_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 7).sum(axis=1).values,
        "promo_14_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=16), 15, 14).sum(axis=1).values,
    }

    for i in [3, 7, 14, 30, 60, 140]:
        tmp1 = get_timespan(df, t2017, i, i)
        tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1

        X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values
        X['has_promo_mean_%s_decay' % i] = (tmp1 * tmp2.replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values

        X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values
        X['no_promo_mean_%s_decay' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values

    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s' % i] = tmp.mean(axis=1).values
        X['median_%s' % i] = tmp.median(axis=1).values
        X['min_%s' % i] = tmp.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        X['std_%s' % i] = tmp.std(axis=1).values

    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017 + timedelta(days=-7), i, i)
        X['diff_%s_mean_2' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values
        X['mean_%s_2' % i] = tmp.mean(axis=1).values
        X['median_%s_2' % i] = tmp.median(axis=1).values
        X['min_%s_2' % i] = tmp.min(axis=1).values
        X['max_%s_2' % i] = tmp.max(axis=1).values
        X['std_%s_2' % i] = tmp.std(axis=1).values

    for i in [7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

        tmp = get_timespan(promo_df, t2017, i, i)
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

    tmp = get_timespan(promo_df, t2017 + timedelta(days=16), 15, 15)
    X['has_promo_days_in_after_15_days'] = (tmp > 0).sum(axis=1).values
    X['last_has_promo_day_in_after_15_days'] = i - ((tmp > 0) * np.arange(15)).max(axis=1).values
    X['first_has_promo_day_in_after_15_days'] = ((tmp > 0) * np.arange(15, 0, -1)).max(axis=1).values

    for i in range(1, 16):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values

    for i in range(-16, 16):
        X["promo_{}".format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)

    X = pd.DataFrame(X)

    if is_train:
        y = df[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X

In [ ]:
print("Preparing dataset...")
t2017 = date(2017, 6, 14)
num_days = 6
X_l, y_l = [], []
for i in range(num_days):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)

    X_tmp2 = prepare_dataset(df_2017_item, promo_2017_item, t2017 + delta, is_train=False, name_prefix='item')
    X_tmp2.index = df_2017_item.index
    X_tmp2 = X_tmp2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

    X_tmp3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, t2017 + delta, is_train=False, name_prefix='store_class')
    X_tmp3.index = df_2017_store_class.index
    X_tmp3 = X_tmp3.reindex(df_2017_store_class_index).reset_index(drop=True)

    X_tmp = pd.concat([X_tmp, X_tmp2, X_tmp3, items.reset_index(), stores.reset_index()], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)

    del X_tmp2
    gc.collect()

X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

del X_l, y_l
X_val, y_val = prepare_dataset(df_2017, promo_2017, date(2017, 7, 26))

X_val2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 7, 26), is_train=False, name_prefix='item')
X_val2.index = df_2017_item.index
X_val2 = X_val2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_val3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 7, 26), is_train=False, name_prefix='store_class')
X_val3.index = df_2017_store_class.index
X_val3 = X_val3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_val = pd.concat([X_val, X_val2, X_val3, items.reset_index(), stores.reset_index()], axis=1)

X_test = prepare_dataset(df_2017, promo_2017, date(2017, 8, 16), is_train=False)

X_test2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 8, 16), is_train=False, name_prefix='item')
X_test2.index = df_2017_item.index
X_test2 = X_test2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_test3 = prepare_dataset(df_2017_store_class, df_2017_promo_store_class, date(2017, 8, 16), is_train=False, name_prefix='store_class')
X_test3.index = df_2017_store_class.index
X_test3 = X_test3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_test = pd.concat([X_test, X_test2, X_test3, items.reset_index(), stores.reset_index()], axis=1)

del X_test2, X_val2, df_2017_item, promo_2017_item, df_2017_store_class, df_2017_promo_store_class, df_2017_store_class_index
gc.collect()